## **Measuring Bifurcation angles from case Normal 1, Normal 7, Diseased 7, Diseased 9**

Calcular el valor dels l'angles de es bifurcacions ja donades pels .txt

Al final d'aquest .ipynb hi ha les bifurcacions calculades en general dels 4 casos de test tal i com venen les dades amb l'output de mimics

### **Imports and Installs**

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
import plotly.graph_objects as go
import networkx as nx
from scipy.spatial.distance import cdist

### **Google Drive connection**

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Functions**

In [ ]:
def parse_and_export_centerlines(file_path):
    branches = defaultdict(list)  # Diccionario donde la clave es el ID de la rama
    current_branch = None  # Guarda el ID de la rama actual
    connections = defaultdict(set)  # Almacena conexiones entre segmentos

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Detectar una nueva rama
            branch_match = re.match(r"\[New Branch Set\] Branch Segment (\d+):", line)
            if branch_match:
                current_branch = int(branch_match.group(1))
                continue

            # Extraer conexiones entre segmentos
            if "connects to:" in line or "connects from:" in line:
                next_line = next(file).strip()
                segments = re.findall(r"Branch Segment (\d+)", next_line)
                connected_segments = set(map(int, segments))
                connections[current_branch].update(connected_segments)
                continue

            # Extraer puntos de la rama actual
            if re.match(r"^\s*-?\d+\.\d+", line):
                values = list(map(float, line.split()[:6]))  # Tomar Px, Py, Pz, Tx, Ty, Tz
                branches[current_branch].append(values)

    # Crear lista de datos para el DataFrame
    all_data = []
    for branch_id, points in branches.items():
        for point in points:
            all_data.append([branch_id] + point)

    columns = ["Branch ID", "Px", "Py", "Pz", "Tx", "Ty", "Tz"]
    df = pd.DataFrame(all_data, columns=columns)
    return df, connections

In [ ]:
def calculate_bifurcation_angles(df, connections):
    angles_data = {}
    for seg1, connected_segments in connections.items():
        for seg2 in connected_segments:
            t1 = df[df["Branch ID"] == seg1][["Tx", "Ty", "Tz"]].values[0]
            t2 = df[df["Branch ID"] == seg2][["Tx", "Ty", "Tz"]].values[0]

            # Calcular el ángulo con el producto escalar
            cos_theta = np.dot(t1, t2) / (np.linalg.norm(t1) * np.linalg.norm(t2))
            angle = np.arccos(np.clip(cos_theta, -1.0, 1.0)) * 180 / np.pi

            angles_data[f"{seg1}-{seg2}"] = angle

    df_angles = pd.DataFrame([angles_data])
    return df_angles

In [ ]:
file_path_rca= '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Normal_1/rca_centerline.txt'
file_path_lca= '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Normal_1/lca_centerline.txt'

In [ ]:
# Ruta del archivo
df, connections = parse_and_export_centerlines(file_path_rca)
df_angles = calculate_bifurcation_angles(df, connections)
df_angles.head()

,1-2,2-1,5-1
0,85.420799,85.420799,28.105592


### **Test using segments pairs**

Utilitzant els dot products dels punts més propers entre segments

In [ ]:
# Pares de segmentos a comparar
bifurcation_pairs_rca = [("3", "4"), ("3", "16"), ("4", "16")]
bifurcation_pairs_lca = [("5", "6"), ("5", "11"), ("6", "9"), ("7", "10"), ("11", "12"), ("13", "14")]

In [ ]:
def calculate_angles_between_segments(segment_pairs, file_paths):
    """
    Calcula el ángulo entre segmentos consecutivos basado en los puntos más cercanos.

    Parameters:
    - segment_pairs: Lista de tuplas con pares de segmentos a comparar.
    - file_paths: Diccionario con rutas de archivos para cada segmento.

    Returns:
    - DataFrame con información de los ángulos entre segmentos.
    """
    angle_data = []

    for seg1, seg2 in segment_pairs:
        if seg1 in file_paths and seg2 in file_paths:
            df1 = parse_and_export_centerlines(file_paths[seg1])
            df2 = parse_and_export_centerlines(file_paths[seg2])

            # Extraer puntos centrales
            points1 = df1[["Px", "Py", "Pz"]].values
            points2 = df2[["Px", "Py", "Pz"]].values

            # Calcular distancias entre todos los pares de puntos
            distances = cdist(points1, points2, metric='euclidean')
            min_idx = np.unravel_index(np.argmin(distances), distances.shape)

            # Obtener los puntos más cercanos
            p1 = points1[min_idx[0]]
            p2 = points2[min_idx[1]]

            # Obtener los vectores tangentes
            t1 = df1.iloc[min_idx[0]][["Tx", "Ty", "Tz"]].values
            t2 = df2.iloc[min_idx[1]][["Tx", "Ty", "Tz"]].values

            # Normalizar vectores
            t1 /= np.linalg.norm(t1)
            t2 /= np.linalg.norm(t2)

            # Calcular ángulo usando producto punto
            dot_product = np.dot(t1, t2)
            angle = np.arccos(np.clip(dot_product, -1.0, 1.0)) * 180 / np.pi

            # Almacenar resultados
            angle_data.append({
                "Segment 1": seg1,
                "Segment 2": seg2,
                "Px1": p1[0], "Py1": p1[1], "Pz1": p1[2],
                "Px2": p2[0], "Py2": p2[1], "Pz2": p2[2],
                "Angle (degrees)": angle
            })
        else:
            print(f"Segment {seg1} or {seg2} not found in file paths.")

    return pd.DataFrame(angle_data)


In [ ]:
df = calculate_angles_between_segments(bifurcation_pairs_rca, file_paths_rca_1)

In [ ]:
df

,Segment 1,Segment 2,Px1,Py1,Pz1,Px2,Py2,Pz2,Angle (degrees)
0,3,4,87.1985,103.6161,26.7149,87.3098,103.6067,26.6271,85.017104
1,3,16,87.1985,103.6161,26.7149,87.1985,103.6161,26.7149,0.762551
2,4,16,87.3098,103.6067,26.6271,87.3098,103.6067,26.6271,0.783996


### **Test using 4 cases: Normal_1, Normal_7, Diseased_7 and Diseased_9**

##### General bifurcations

Calcular el grau de les bifurcacions sense tenir en compte numeració correcte. Simplement tal i com venen.

In [ ]:
def parse_and_export_centerlines(file_path):
    branches = defaultdict(list)
    current_branch = None
    connections = defaultdict(set)

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            branch_match = re.match(r"\[New Branch Set\] Branch Segment (\d+):", line)
            if branch_match:
                current_branch = int(branch_match.group(1))
                continue

            if "connects to:" in line or "connects from:" in line:
                next_line = next(file).strip()
                segments = re.findall(r"Branch Segment (\d+)", next_line)
                connected_segments = set(map(int, segments))
                connections[current_branch].update(connected_segments)
                continue

            if re.match(r"^\s*-?\d+\.\d+", line):
                values = list(map(float, line.split()[:6]))
                branches[current_branch].append(values)

    all_data = []
    for branch_id, points in branches.items():
        for point in points:
            all_data.append([branch_id] + point)

    columns = ["Branch ID", "Px", "Py", "Pz", "Tx", "Ty", "Tz"]
    df = pd.DataFrame(all_data, columns=columns)
    return df, connections

def calculate_bifurcation_angles(df, connections, case_name):
    angles_data = []
    for seg1, connected_segments in connections.items():
        for seg2 in connected_segments:
            t1 = df[df["Branch ID"] == seg1][["Tx", "Ty", "Tz"]].values[0]
            t2 = df[df["Branch ID"] == seg2][["Tx", "Ty", "Tz"]].values[0]

            cos_theta = np.dot(t1, t2) / (np.linalg.norm(t1) * np.linalg.norm(t2))
            angle = np.arccos(np.clip(cos_theta, -1.0, 1.0)) * 180 / np.pi

            angles_data.append([case_name, f"{seg1}-{seg2}", angle])

    return angles_data

In [ ]:

cases = ["Normal_1", "Normal_7", "Diseased_7", "Diseased_9"]
all_angles = []

for case in cases:
    for artery in ['rca', 'lca']:
        file_path = f'/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/{case}/{artery}_centerline.txt'
        df, connections = parse_and_export_centerlines(file_path)
        all_angles.extend(calculate_bifurcation_angles(df, connections, case))

df_angles = pd.DataFrame(all_angles, columns=["Case", "Segment a - b", "Angle"])

In [ ]:
df_angles

,Case,Segment a - b,Angle
0,Normal_1,16-3,85.420799
1,Normal_1,3-16,85.420799
2,Normal_1,4-16,28.105592
3,Normal_1,10-8,41.812245
4,Normal_1,13-12,115.606384
...,...,...,...
57,Diseased_9,9-7,82.152738
58,Diseased_9,10-8,110.815420
59,Diseased_9,12-11,83.936782
60,Diseased_9,11-11,0.000000


In [ ]:
# Reformatear los nombres de las columnas para que tengan el formato adecuado
df_angles["Segment a - b"] = df_angles["Segment a - b"].apply(lambda x: f"{'RCA' if 'rca' in x else 'LCA'}_{tuple(map(int, x.split('-')))}")

# Pivotear el DataFrame para obtener la estructura deseada
df_pivot = df_angles.pivot(index="Case", columns="Segment a - b", values="Angle")

# Resetear el índice para que "Case" vuelva a ser una columna
df_pivot.reset_index(inplace=True)

In [ ]:
df_pivot

Segment a - b,Case,"LCA_(10, 10)","LCA_(10, 8)","LCA_(11, 11)","LCA_(11, 12)","LCA_(11, 17)","LCA_(11, 5)","LCA_(11, 6)","LCA_(12, 11)","LCA_(13, 12)",...,"LCA_(6, 7)","LCA_(6, 9)","LCA_(7, 6)","LCA_(7, 7)","LCA_(7, 8)","LCA_(7, 9)","LCA_(8, 10)","LCA_(8, 7)","LCA_(9, 7)","LCA_(9, 9)"
0,Diseased_7,0.0,139.599917,NaN,NaN,NaN,95.695585,NaN,NaN,NaN,...,NaN,23.178031,NaN,0.000001,60.224345,12.454346,NaN,60.224345,12.454346,0.0
1,Diseased_9,NaN,110.815420,0.0,NaN,NaN,NaN,71.512234,83.936782,NaN,...,35.687899,NaN,35.687899,NaN,101.356757,NaN,110.815420,NaN,82.152738,NaN
2,Normal_1,NaN,41.812245,NaN,96.811280,160.699526,NaN,63.385116,96.811280,115.606384,...,NaN,49.792321,NaN,NaN,72.110033,72.177421,41.812245,NaN,72.177421,NaN
3,Normal_7,NaN,NaN,NaN,66.595405,150.477775,NaN,116.900684,66.595405,81.866411,...,89.143653,NaN,89.143653,NaN,NaN,NaN,NaN,NaN,64.791691,NaN


In [ ]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict

# Pares de segmentos a comparar
bifurcation_pairs_rca = [("3", "4"), ("3", "16"), ("4", "16")]
bifurcation_pairs_lca = [("5", "6"), ("5", "11"), ("6", "9"), ("7", "10"), ("11", "12"), ("13", "14")]

def parse_and_export_centerlines(file_path):
    branches = defaultdict(list)
    current_branch = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            branch_match = re.match(r"\[New Branch Set\] Branch Segment (\d+):", line)
            if branch_match:
                current_branch = int(branch_match.group(1))
                continue

            if re.match(r"^\s*-?\d+\.\d+", line):
                values = list(map(float, line.split()[:6]))
                branches[current_branch].append(values)

    all_data = []
    for branch_id, points in branches.items():
        for point in points:
            all_data.append([branch_id] + point)

    columns = ["Branch ID", "Px", "Py", "Pz", "Tx", "Ty", "Tz"]
    df = pd.DataFrame(all_data, columns=columns)
    return df

def calculate_bifurcation_angles(df, bifurcation_pairs, artery):
    angles_data = {}
    for seg1, seg2 in bifurcation_pairs:
        try:
            t1 = df[df["Branch ID"] == int(seg1)][["Tx", "Ty", "Tz"]].values[0]
            t2 = df[df["Branch ID"] == int(seg2)][["Tx", "Ty", "Tz"]].values[0]

            cos_theta = np.dot(t1, t2) / (np.linalg.norm(t1) * np.linalg.norm(t2))
            angle = np.arccos(np.clip(cos_theta, -1.0, 1.0)) * 180 / np.pi

            connection_key = f"{artery}_({seg1}, {seg2})"
            angles_data[connection_key] = angle
        except IndexError:
            # Handle missing segments gracefully
            connection_key = f"{artery}_({seg1}, {seg2})"
            angles_data[connection_key] = np.nan

    return angles_data

cases = ["Normal_1", "Normal_7", "Diseased_7", "Diseased_9"]
columns = ["case"]
data = []

for case in cases:
    case_data = {"case": case}
    for artery, bifurcation_pairs in zip(['RCA', 'LCA'], [bifurcation_pairs_rca, bifurcation_pairs_lca]):
        file_path = f'/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/{case}/{artery.lower()}_centerline.txt'
        df = parse_and_export_centerlines(file_path)
        angles = calculate_bifurcation_angles(df, bifurcation_pairs, artery)
        case_data.update(angles)

        # Add new connections to columns if they don't exist
        for connection in angles.keys():
            if connection not in columns:
                columns.append(connection)

    data.append(case_data)

# Ensure all rows have the same columns, filling missing values with NaN
df_result = pd.DataFrame(data, columns=columns).fillna(np.nan)

# Mostrar el DataFrame final
df_result


,case,"RCA_(3, 4)","RCA_(3, 16)","RCA_(4, 16)","LCA_(5, 6)","LCA_(5, 11)","LCA_(6, 9)","LCA_(7, 10)","LCA_(11, 12)","LCA_(13, 14)"
0,Normal_1,68.138373,85.420799,28.105592,50.854250,56.692442,49.792321,35.373625,96.811280,NaN
1,Normal_7,92.177424,38.328470,129.686035,52.502144,84.544581,100.013332,NaN,66.595405,93.441697
2,Diseased_7,88.800416,85.094490,142.694608,49.089263,95.695585,23.178031,93.118395,NaN,NaN
3,Diseased_9,107.970671,95.423280,14.608360,33.609627,90.171300,106.480215,21.200389,83.936782,NaN
